# Preparing for colab

In [1]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && cp /content/drive/MyDrive/backups/kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Mounted at /content/drive


In [3]:
!kaggle competitions download -c amex-default-prediction -f sample_submission.csv
!unzip /content/sample_submission.csv.zip

 31% 10.0M/32.4M [00:00<00:00, 104MB/s]
100% 32.4M/32.4M [00:00<00:00, 202MB/s]
Archive:  /content/sample_submission.csv.zip
  inflating: sample_submission.csv   


In [4]:
!kaggle competitions download -c amex-default-prediction -f train_labels.csv
!unzip /content/train_labels.csv.zip

 62% 10.0M/16.2M [00:00<00:00, 103MB/s]
100% 16.2M/16.2M [00:00<00:00, 143MB/s]
Archive:  /content/train_labels.csv.zip
  inflating: train_labels.csv        


In [5]:
!kaggle datasets download -d raddar/amex-data-integer-dtypes-parquet-format
!unzip /content/amex-data-integer-dtypes-parquet-format.zip

100% 4.06G/4.07G [00:19<00:00, 255MB/s]
100% 4.07G/4.07G [00:19<00:00, 221MB/s]
Archive:  /content/amex-data-integer-dtypes-parquet-format.zip
  inflating: test.parquet            
  inflating: train.parquet           


# Create Dataset

In [6]:
# ====================================================
# Library
# ====================================================
import gc; gc.enable()
import warnings
warnings.filterwarnings('ignore')
import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
import itertools

def get_difference(data, num_features):
    df1 = []
    customer_ids = []
    for customer_id, df in tqdm(data.groupby(['customer_ID'])):
        diff_df1 = df[num_features].diff(1).iloc[[-1]].values.astype(np.float32)
        df1.append(diff_df1)
        customer_ids.append(customer_id)
    df1 = np.concatenate(df1, axis = 0)
    df1 = pd.DataFrame(df1, columns = [col + '_diff1' for col in df[num_features].columns])
    df1['customer_ID'] = customer_ids
    return df1

# ====================================================
# Read & preprocess data and save it to disk
# ====================================================
def read_preprocess_data():
    train = pd.read_parquet('/content/train.parquet')
    features = train.drop(['customer_ID', 'S_2'], axis = 1).columns.to_list()
    cat_features = [
        "B_30",
        "B_38",
        "D_114",
        "D_116",
        "D_117",
        "D_120",
        "D_126",
        "D_63",
        "D_64",
        "D_66",
        "D_68",
    ]
    num_features = [col for col in features if col not in cat_features]

    # train.S_2 = pd.to_datetime(train.S_2)
    # train_s2_agg = train.groupby("customer_ID")['S_2'].agg(['first', 'last'])
    # train_s2_agg = (train_s2_agg['last'] - train_s2_agg['first']).dt.days
    # train_s2_agg = train_s2_agg.to_frame()
    # train_s2_agg.columns = [ 'S_2_lag']
    # train_s2_agg.reset_index(inplace=True)
    
    # Train FE
    print('Starting train feature extraction')
    train_num_agg = train.groupby("customer_ID")[num_features].agg(['first', 'mean', 'std', 'min', 'max', 'last'])
    train_num_agg.columns = ['_'.join(x) for x in train_num_agg.columns]
    train_num_agg.reset_index(inplace = True)

    # Lag Features
    for col in train_num_agg:
        for col_2 in ['first', 'mean', 'std', 'min', 'max']:
            if 'last' in col and col.replace('last', col_2) in train_num_agg:
                train_num_agg[col + '_lag_sub'] = train_num_agg[col] - train_num_agg[col.replace('last', col_2)]

    train_cat_agg = train.groupby("customer_ID")[cat_features].agg(['count', 'first', 'last', 'nunique'])
    train_cat_agg.columns = ['_'.join(x) for x in train_cat_agg.columns]
    train_cat_agg.reset_index(inplace = True)
    
    # Transform float64 columns to float32
    cols = list(train_num_agg.dtypes[train_num_agg.dtypes == 'float64'].index)
    for col in tqdm(cols):
        train_num_agg[col] = train_num_agg[col].astype(np.float32)
    # Transform int64 columns to int32
    cols = list(train_cat_agg.dtypes[train_cat_agg.dtypes == 'int64'].index)
    for col in tqdm(cols):
        train_cat_agg[col] = train_cat_agg[col].astype(np.int32)

    train_labels = pd.read_csv('/content/train_labels.csv')
    
    # Get the difference
    train_diff = get_difference(train, num_features)
    train = train_num_agg.merge(train_cat_agg, how = 'inner', on = 'customer_ID').merge(train_diff, how = 'inner', on = 'customer_ID').merge(train_labels, how = 'inner', on = 'customer_ID')

    print('Train shape: ', train.shape)
    del train_num_agg, train_cat_agg, train_diff
    gc.collect() 

    train.to_parquet('train_fe_plus_plus.parquet')
    del train
    gc.collect()
    
    # Test FE
    test = pd.read_parquet('/content/test.parquet')
    print('Starting test feature extraction')

    # test.S_2 = pd.to_datetime(test.S_2)
    # test_s2_agg = test.groupby("customer_ID")['S_2'].agg(['first', 'last'])
    # test_s2_agg = (test_s2_agg['last'] - test_s2_agg['first']).dt.days
    # test_s2_agg = test_s2_agg.to_frame()
    # test_s2_agg.columns = [ 'S_2_lag']
    # test_s2_agg.reset_index(inplace=True)

    test_num_agg = test.groupby("customer_ID")[num_features].agg(['first', 'mean', 'std', 'min', 'max', 'last'])
    test_num_agg.columns = ['_'.join(x) for x in test_num_agg.columns]
    test_num_agg.reset_index(inplace = True)

    # Lag Features
    for col in test_num_agg:
        for col_2 in ['first', 'mean', 'std', 'min', 'max']:
                    if 'last' in col and col.replace('last', col_2) in test_num_agg:
                        test_num_agg[col + '_lag_sub'] = test_num_agg[col] - test_num_agg[col.replace('last', col_2)]
    
    test_cat_agg = test.groupby("customer_ID")[cat_features].agg(['count', 'first', 'last', 'nunique'])
    test_cat_agg.columns = ['_'.join(x) for x in test_cat_agg.columns]
    test_cat_agg.reset_index(inplace = True)

    # Transform float64 columns to float32
    cols = list(test_num_agg.dtypes[test_num_agg.dtypes == 'float64'].index)
    for col in tqdm(cols):
        test_num_agg[col] = test_num_agg[col].astype(np.float32)
    # Transform int64 columns to int32
    cols = list(test_cat_agg.dtypes[test_cat_agg.dtypes == 'int64'].index)
    for col in tqdm(cols):
        test_cat_agg[col] = test_cat_agg[col].astype(np.int32)

    # Get the difference
    test_diff = get_difference(test, num_features)
    test = test_num_agg.merge(test_cat_agg, how = 'inner', on = 'customer_ID').merge(test_diff, how = 'inner', on = 'customer_ID')

    print('Test shape: ', test.shape)
    del test_num_agg, test_cat_agg, test_diff
    gc.collect()
    
    
    # Save files to disk
    test.to_parquet('test_fe_plus_plus.parquet')
    del test
    gc.collect()
    
# Read & Preprocess Data
read_preprocess_data()

Starting train feature extraction


  0%|          | 0/261 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/458913 [00:00<?, ?it/s]

Train shape:  (458913, 1462)
Starting test feature extraction


  0%|          | 0/261 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/924621 [00:00<?, ?it/s]

Test shape:  (924621, 1461)


In [7]:
!mkdir amex-fe-plus3

In [8]:
!mv /content/train_fe_plus_plus.parquet /content/amex-fe-plus3/
!mv /content/test_fe_plus_plus.parquet /content/amex-fe-plus3/

In [9]:
!kaggle datasets init -p /content/amex-fe-plus3

Data package template written to: /content/amex-fe-plus3/dataset-metadata.json


In [10]:
%%writefile /content/amex-fe-plus3/dataset-metadata.json
{
  "title": "Amex-FE-Plus3",
  "id": "ryuina/amex-fe-plus3",
  "licenses": [
    {
      "name": "CC0-1.0"
    }
  ]
}

Overwriting /content/amex-fe-plus3/dataset-metadata.json


In [11]:
# !kaggle datasets version -p /content/amex-fe-plus -m "Created only last - first features"

# for the first time
!kaggle datasets create -p /content/amex-fe-plus3

Starting upload for file train_fe_plus_plus.parquet
100% 1.38G/1.38G [00:20<00:00, 73.2MB/s]
Upload successful: train_fe_plus_plus.parquet (1GB)
Starting upload for file test_fe_plus_plus.parquet
100% 2.49G/2.49G [00:28<00:00, 95.2MB/s]
Upload successful: test_fe_plus_plus.parquet (2GB)
Your private Dataset is being created. Please check progress at https://www.kaggle.com/datasets/ryuina/amex-fe-plus3
